In [ ]:
import numpy as np
import pandas as pd
import cv2
import os
import random

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

import sys
import config

sys.path.append('model_building/create_image_folders.py')
from model_building.create_image_folders import * 
sys.path.append('model_building/cnn_model_keras.py')
from model_building.cnn_model_keras import *

import warnings
warnings.simplefilter("ignore", UserWarning)


## Load Data

In [ ]:
## Set paths
img_folder = "/home/jovyan/my_work/QB/image/images/"
train_img = "/home/jovyan/my_work/QB/image/train/"
val_img = "/home/jovyan/my_work/QB/image/val/"
labels_image = "data/x-ai_data.csv"

## Create Subfolder

Use the fonction "subfolders" to do the same thing as the cells below, DO NOT run it twice to avoid duplicates

In [ ]:
labels_image = pd.read_csv(labels_image)
train = labels_image.loc[labels_image['split']=='train']
val = labels_image.loc[labels_image['split']=='validation']

In [ ]:
train0 = train.loc[train['class']==0]
train1 = train.loc[train['class']==1]

val0 = val.loc[val['class']==0]
val1 = val.loc[val['class']==1]


In [ ]:
#Move TRAIN images labeled 0 to the correct folder

for i in train0.index:
    im = cv2.imread(img_folder+train0.loc[i,'filename'])
    cv2.imwrite(train_img + '0/' +train0.loc[i,'filename'], im)
#Move TRAIN images labeled 1 to the correct folder
for i in train1.index:
    im = cv2.imread(img_folder+train1.loc[i,'filename'])
    cv2.imwrite(train_img+'1/' +train1.loc[i,'filename'], im)

In [ ]:
#Move val images labeled 0 to the correct folder
for i in val0.index:
    im = cv2.imread(img_folder+val0.loc[i,'filename'])
    cv2.imwrite(val_img + '0/' +val0.loc[i,'filename'], im)
#Move val images labeled 1 to the correct folder
for i in val1.index:
    im = cv2.imread(img_folder+val1.loc[i,'filename'])
    cv2.imwrite(val_img+'1/' +val1.loc[i,'filename'], im)

In [ ]:
import shutil

#Remove hidden file
shutil.rmtree("image/train/.ipynb_checkpoints")
shutil.rmtree("image/val/.ipynb_checkpoints")

## Model

In [ ]:
## Train and Val dataset
train_ds = train_set(train_img, config.image_size, config.batch_size)
val_ds = val_set(val_img, config.image_size, config.batch_size)

In [ ]:
plt.figure(figsize=(10, 10))
for images, labels in train_ds.take(1):
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title(int(labels[i]))
        plt.axis("off")

In [ ]:
## Model
model = make_model(input_shape=config.image_size + (3,), num_classes=2)

In [ ]:
train_model(model, train_ds, val_ds, 2)

In [ ]:
model.save("model2.h5")

In [ ]:
#visualize data augmentation
data_augmentation = keras.Sequential(
    [
        layers.RandomFlip("horizontal_and_vertical"),
        layers.RandomRotation(0.1),
        layers.RandomContrast([0,1]),
        layers.RandomTranslation(height_factor=0.2, width_factor=0.2)
    ]
)
plt.figure(figsize=(10, 10))
for images, _ in train_ds.take(1):
    for i in range(9):
        augmented_images = data_augmentation(images)
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(augmented_images[0].numpy().astype("uint8"))
        plt.axis("off")

## try test set

In [ ]:
labels_image = pd.read_csv(labels_image)

test = labels_image.loc[labels_image['split']=='test']
test

In [ ]:
test_img = "/home/jovyan/my_work/QB/image/test/"
#Inside the test folder create another test folder

In [ ]:
#Move TEST images to the correct folder
for i in test.index:
    im = cv2.imread(img_folder+test.loc[i,'filename'])
    cv2.imwrite(test_img + 'test/' + test.loc[i,'filename'], im)

In [ ]:
import shutil

shutil.rmtree("image/test/.ipynb_checkpoints")

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense

model = Sequential()
model.add(Conv2D(32, (2, 2), input_shape=(256,256,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
  
model.add(Conv2D(32, (2, 2)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
  
model.add(Conv2D(64, (2, 2)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
  
model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

In [ ]:
model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)
  
test_datagen = ImageDataGenerator(rescale=1. / 255)

In [ ]:
train_generator = train_datagen.flow_from_directory(
    train_img,
    target_size=(256, 256),
    batch_size=config.batch_size,
    class_mode='binary')

In [ ]:
validation_generator = test_datagen.flow_from_directory(
    val_img,
    target_size=(256, 256),
    batch_size=config.batch_size,
    class_mode='binary')

In [ ]:
model.fit_generator(
    train_generator,
    steps_per_epoch=1400 // config.batch_size,
    epochs=5,
    validation_data=validation_generator,
    validation_steps=400 // config.batch_size)

In [ ]:
from tensorflow.keras.models import load_model

model.save("model3.h5")
model3 = load_model("model3.h5")

In [ ]:
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array

image = load_img('my_work/QB/image/test/silos_256-0-0--6-16-472-29195.png', target_size=(256, 256))
img = np.array(image)
img = img / 255.0
img = img.reshape(1,256,256,3)
label = model.predict(img)
print("Predicted Class (0 - No Silo , 1- Silo): ", label[0][0])

In [ ]:
test_generator = test_datagen.flow_from_directory(
    test_img,
    target_size=(256, 256),
    batch_size=config.batch_size,
    class_mode=None,
    shuffle=False
    )

In [ ]:
test_generator.reset()

In [ ]:
predictions = model3.predict_generator(test_generator,verbose=1,steps=200/config.batch_size)
predictions

In [ ]:
predicted_class_indices= np.round(predictions).astype(int)

In [ ]:
labels = (train_generator.class_indices)
labels = dict((v,k) for k,v in labels.items())
predictions = [labels[k] for k in predicted_class_indices[:,0]]

In [ ]:
filenames=test_generator.filenames
results=pd.DataFrame({"Filename":filenames,
                      "Predictions":predictions})

In [ ]:
results